In [20]:
import numpy as np
import k3d
from k3d.platonic import Cube
from ratelimiter import RateLimiter

In [21]:
plot = k3d.plot(camera_auto_fit=False, grid_auto_fit=False, grid=(-5,-5,-5,5,5,5))

cube = Cube(origin=[0,0,0], size=5).mesh
cube.wireframe = True
plot += cube

In [22]:
class Ball:
    def __init__(self, position=[0,0,0], velocity=[0,0,0], random=False):
        self.position = np.array(position, dtype=np.float32)
        self.velocity = np.array(velocity, dtype=np.float32)
        
        if random:
            a, b = -3, 4
            self.position = ((b - a) * np.random.random(size=(3)) + a).astype(np.float32)
            self.velocity = np.random.uniform(-4, 4, size=(3)).astype(np.float32)

In [23]:
n = 50
ball_scenes = []

balls = [Ball(random=True) for i in range(n)]
balls_position = np.array([ball.position for ball in balls])
balls_velocity = np.array([ball.velocity for ball in balls])

k3dballs = k3d.points(balls_position, point_size=0.3, shader='simple', color=0)
plot += k3dballs

In [24]:
g = np.array([0, 0, -9.81], dtype=np.float32)
time_steps, dt = 1200, 0.01

# simulation
for t in range(1, time_steps):
        balls_position = balls_position + balls_velocity*dt + 0.5*g*dt**2
        ball_scenes.append(balls_position.copy())
        balls_velocity = balls_velocity + g*dt  

        balls_velocity[balls_position[:,2] - (-4) < 0.3] *= np.array([1, 1, -1])
        balls_velocity[balls_position[:,2] - (4) > 0.3] *= np.array([1, 1, -1])
            
        balls_velocity[balls_position[:,1] - (-4) < 0.3] *= np.array([1, -1, 1])
        balls_velocity[balls_position[:,1] - (4) > 0.3] *= np.array([1, -1, 1])

        balls_velocity[balls_position[:,0] - (-4) < 0.3] *= np.array([-1, 1, 1])
        balls_velocity[balls_position[:,0] - (4) > 0.3] *= np.array([-1, 1, 1])
        
ball_scenes = np.array(ball_scenes)

In [25]:
plot.display()

Output()

In [26]:
rate_limiter = RateLimiter(max_calls=1, period=1/20)

# visualisation
for t in range(time_steps-1):
    with rate_limiter:
        k3dballs.positions = ball_scenes[t]